In [77]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import pandas as pd
pd.options.display.max_rows = 5000 #(设定能显示的行数)

###### Main cities in the UK

In [3]:
cities = 'London ,Manchester,Birmingham,Wolverhampton ,Leeds, Bradford ,Glasgow ,Southampton,Portsmouth ,Liverpool ,Newcastle ,Nottingham ,Sheffield,Bristol,Belfast,Leicester'
city_lst = []
for i in cities.split(','):
    city_lst.append(i.strip())
    
print(city_lst)

['London', 'Manchester', 'Birmingham', 'Wolverhampton', 'Leeds', 'Bradford', 'Glasgow', 'Southampton', 'Portsmouth', 'Liverpool', 'Newcastle', 'Nottingham', 'Sheffield', 'Bristol', 'Belfast', 'Leicester']


##### Checking requests status here

In [4]:
url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start=0'
# url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start=10' # worked fine
r = requests.get(url)
r.status_code

200

In [5]:
# Create soup and have a glimpse
soup = BeautifulSoup(r.text, 'html.parser')
# print(soup.prettify())

#### 1.注意这里的 page 变量的使用。
#### 2.整个过程其实可以写成一个function。

##### ？小问题，salary在为空时不会反悔任何内容，这就让title，location，salary之后的排序产生问题

In [73]:
titles = []
companies = []
# locations = []  # sequence doesn't match others
summaries = []
for page in range(0, 200, 10):
    url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start={}'.format(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for title in soup.find_all('a', attrs={'data-tn-element':'jobTitle'}):
        try:
            titles.append(title.text.strip())
        except:
            titles.append('Nothing')
    for company in soup.find_all('span', attrs={'class':'company'}):
        try:
            companies.append(company.text.strip())
        except:
            companies.append('Nothing')
#     for location in soup.find_all('span', attrs={'class':'location'}):
#         try:
#             locations.append(location.text.strip())
#         except:
#             locations.append('Nothing')
    for summary in soup.find_all("div",{"class":"summary"}):
        try:
            summaries.append(summary.text.strip())
        except:
            summaries.append('Nothing')

In [74]:
print(len(titles))
print(len(companies))
#print(len(locations))
print(len(summaries))

280
280
280


In [75]:
S_results = pd.DataFrame({'title':titles, 'company':companies, 'summary':summaries})

In [78]:
S_results

,title,company,summary
0,Junior Data Scientist,Novafutur,Being a data scientist would require the abili...
1,Junior Data Analyst,Smarkets,"You enjoy collaborating with data scientists, ..."
2,Junior Data Scientist,Fable Data Limited,"We need to understand Partner Specific data, M..."
3,Sr. Data Scientist - ML Labs,AWS EMEA SARL (UK Branch),Strong communication and data presentation ski...
4,Data Scientist,Amazon Digital UK Limited,Experience with data scripting languages (e.g....
5,Junior Data Scientist - London,Echobox,"Working knowledge of big data architectures, i..."
6,Junior Marketing Executive,Smarkets,"As a Junior marketing exec at Smarkets, you wi..."
7,Junior Land Consultant,WSP,An understanding of data security protocols.\n...
8,Senior Data Scientist - Algorithms,Deliveroo,We are looking for a senior data scientist fro...
9,AI-Quant - Junior,Crypton Partner Management,One+ year of experience with a data analysis a...


In [ ]:
# DF TO EXCEL
from pandas import ExcelWriter

writer = ExcelWriter('PythonExport.xlsx')
yourdf.to_excel(writer,'Sheet5')
writer.save()

# DF TO CSV
yourdf.to_csv('PythonExport.csv', sep=',')

In [79]:
from pandas import ExcelWriter

In [80]:
writer = ExcelWriter('PythonExport.xlsx')
S_results.to_excel(writer,'Sheet5')
writer.save()

salaries = []
for page in range(0, 100, 10):
    url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start={}'.format(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for title in soup.find_all("span","salary no-wrap"):
        try:
            salaries.append(title.text.strip())
        except:
            salaries.append('No content')

print(len(salaries))
print(salaries)

需求：把上面的titles和salaries装入一个pandas表格，让title和salary一一对应。

##  To be solved
##### 我现在能理解，在做之前的project中，for job in soup.find_all('div', attrs={'class':'result'}): 这段代码其实是将每次搜索的范围限制在一个工作内容的卡片中。
##### 这样就能自动匹配所有的title,company, salary等等。
##### 而我目前还没有找到方法，将每一次的搜索限定在一个小卡片中。所以，如果有某个信息缺失时，之后的信息可能发生错位。

In [38]:
titles = []
salaries = []
for page in range(0,100,10):
    url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start={}'.format(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for job in soup.find_all('div', attrs={'class':'jobsearch-SerpJobCard unifiedRow row result clickcard'}):
        try:
            titles.append(job.find('a', attrs={'data-tn-element':'jobTitle'}))
        except:
            titles.append(np.nan)
        
        try:
            salaries.append(job.find("span","salary no-wrap"))
        except:
            salaries.append(np.nan)
                

UK Indeed 的一个卡片网页代码：

<div class="jobsearch-SerpJobCard unifiedRow row result clickcard" id="pj_1efee661bdfabb84" data-jk="1efee661bdfabb84" data-empn="5951703298586437" data-ci="360037745">

<style>
.jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}.jasxrefreshcombotst .jobcard_logo img{max-height:2rem;max-width:100%}
</style>


<h2 class="title">
<a target="_blank" id="sja1" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0DM3XBi8UFNat4xrxKgt90CEA066Pp5zFAvDf-Fs8XmhJTcjHQ7jMqRdWNtbENZjRV9WkydgHWncJ5CoKqvzrslhe_llTJCuEu79pRBDEJ4m1jsjHw76W9m9xg5hQuJMNFHsVES5HdHxCkjFqEykpykvnzeIDVL5kil_CuUinBOX7EBEu3yLVLqi6YKeIdBSsGXGQkZNvjLbVzF0_4iZnCoeNH8yt3hl1-JiQK6XiCk49dMbGhmcB0N8BBvfjbUeNoM4cWkfsfIZZbi-EVkyJ3UfvNfli3sOQGpTTmIVteZKQwVOvsC2n7TdrbmvGVo8CM01xfgEQ86cdYo51SKQdYE8fs5aXUxHfKiwFpi8O2a-i9kmZRwkr6JVjpXW4V5IeJR5DerbNICc6nzMmoRx49D771bqtI_Z2-LBI99mnlwUCA3CavgOZvTt1NlEEBAafC8w8YMrcPODkwtnoTjoI5KraXXoWN6ZDuUnx4tt_AJrCNZOsJlBaIFVihdYLBOYLrenw-t9DSH-A==&amp;p=1&amp;fvj=1&amp;vjs=3" onmousedown="sjomd('sja1'); clk('sja1'); rclk(this,jobmap[1],1);" onclick="setRefineByCookie([]); sjoc('sja1', 1); convCtr('SJ'); rclk(this,jobmap[1],true,1);" rel="noopener nofollow" title="Head of Medical Statistics / Principal Medical Statistician" class="jobtitle turnstileLink " data-tn-element="jobTitle">
Head of Medical Statistics / Principal Medical Statistician</a>

</h2>

<div class="sjcl">
<div>
<span class="company">
PHMR Ltd</span>

</div>
<div id="recJobLoc_1efee661bdfabb84" class="recJobLoc" data-rc-loc="London" style="display: none"></div>
<div class="location accessible-contrast-color-location">London NW1 8XY</div>
</div>

<div class="salarySnippet salarySnippetDemphasizeholisticSalary">
<span class="salary no-wrap">
<span class="salaryText">
£53,150 - £76,017 a year</span>
</span>
</div>
<div class="jobCardReqContainer"><div class="jobCardReqHeader">Requirements</div><div class="jobCardReqList"><div class="jobCardReqItem">Medical statistics to support HTA/HEOR activities: 5 years</div></div></div><table class="jobCardShelfContainer" role="presentation"><tbody><tr class="jobCardShelf"><td class="jobCardShelfItem indeedApply"><span class="jobCardShelfIcon"><svg width="16" height="16" viewBox="0 0 20 20" fill="none"><rect width="20" height="20" rx="10" fill="#FF5A1F"></rect><path fill-rule="evenodd" clip-rule="evenodd" d="M15.3125 4.0625L10.8125 15.3125L7.99999 11.375L15.3125 4.0625ZM7.604 12.7576L6.875 15.3125L8.567 14.1054L7.604 12.7576ZM7.20463 10.5796L12.419 5.36525L4.0625 9.125L6.9875 10.7968L7.20463 10.5796Z" fill="white"></path></svg></span><span class="iaLabel iaIconActive">Apply with your Indeed CV</span></td></tr></tbody></table><div class="summary">
<ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;"> 
 <li style="margin-bottom:0px;">Fluency in English (for writing papers, reports, proposals, giving presentations and mentoring <b>junior</b> staff);</li>
 <li>Good time management and organisational skills.</li>
</ul></div>
<div class="jobsearch-SerpJobCard-footer">
<div class="jobsearch-SerpJobCard-footerActions">

<div class="result-link-bar-container">
<div class="result-link-bar"><span class="date ">8 days ago</span><span id="tt_set_1" class="tt_set"><div class="job-reaction"><button class="job-reaction-kebab" aria-haspopup="true" aria-expanded="false" data-ol-has-click-handler="" tabindex="0" aria-label="save or dislike" onclick="toggleKebabMenu('1efee661bdfabb84', true, event); return false;"></button><span class="job-reaction-kebab-menu"><button class="job-reaction-kebab-item job-reaction-save" onclick="changeJobState('1efee661bdfabb84', 'save', 'linkbar', true, '');return false;" data-ol-has-click-handler=""><svg focusable="false" viewBox="0 0 24 24" height="16" width="16"><g><path fill="#2d2d2d" d="M16.5,3A6,6,0,0,0,12,5.09,6,6,0,0,0,7.5,3,5.45,5.45,0,0,0,2,8.5C2,12.28,5.4,15.36,10.55,20L12,21.35,13.45,20C18.6,15.36,22,12.28,22,8.5A5.45,5.45,0,0,0,16.5,3ZM12.1,18.55l-0.1.1-0.1-.1C7.14,14.24,4,11.39,4,8.5A3.42,3.42,0,0,1,7.5,5a3.91,3.91,0,0,1,3.57,2.36h1.87A3.88,3.88,0,0,1,16.5,5,3.42,3.42,0,0,1,20,8.5C20,11.39,16.86,14.24,12.1,18.55Z"></path></g></svg><span class="job-reaction-kebab-item-text">Save job</span></button><button class="job-reaction-kebab-item job-reaction-dislike" onclick="dislikeJob(false, false, '1efee661bdfabb84', 'unsave', 'linkbar', true, '');" data-ol-has-click-handler=""><span class="job-reaction-dislike-icon"></span><span class="job-reaction-kebab-item-text">Not interested</span></button><button class="job-reaction-kebab-item job-reaction-report" onclick="reportJob('1efee661bdfabb84');"><span class="job-reaction-report-icon"></span><span class="job-reaction-kebab-item-text">Report job</span></button></span></div></span><div id="editsaved2_1efee661bdfabb84" class="edit_note_content" style="display:none;"></div><script>if (!window['sj_result_1efee661bdfabb84']) {window['sj_result_1efee661bdfabb84'] = {};}window['sj_result_1efee661bdfabb84']['showSource'] = false; window['sj_result_1efee661bdfabb84']['source'] = "Indeed"; window['sj_result_1efee661bdfabb84']['loggedIn'] = true; window['sj_result_1efee661bdfabb84']['showMyJobsLinks'] = true;window['sj_result_1efee661bdfabb84']['undoAction'] = "unsave";window['sj_result_1efee661bdfabb84']['relativeJobAge'] = "8 days ago";window['sj_result_1efee661bdfabb84']['jobKey'] = "1efee661bdfabb84"; window['sj_result_1efee661bdfabb84']['myIndeedAvailable'] = true; window['sj_result_1efee661bdfabb84']['showMoreActionsLink'] = window['sj_result_1efee661bdfabb84']['showMoreActionsLink'] || false; window['sj_result_1efee661bdfabb84']['resultNumber'] = 1; window['sj_result_1efee661bdfabb84']['jobStateChangedToSaved'] = false; window['sj_result_1efee661bdfabb84']['searchState'] = "q=junior data scientist&amp;l=london&amp;start=40"; window['sj_result_1efee661bdfabb84']['basicPermaLink'] = "https://www.indeed.co.uk"; window['sj_result_1efee661bdfabb84']['saveJobFailed'] = false; window['sj_result_1efee661bdfabb84']['removeJobFailed'] = false; window['sj_result_1efee661bdfabb84']['requestPending'] = false; window['sj_result_1efee661bdfabb84']['notesEnabled'] = false; window['sj_result_1efee661bdfabb84']['currentPage'] = "serp"; window['sj_result_1efee661bdfabb84']['sponsored'] = true;window['sj_result_1efee661bdfabb84']['reportJobButtonEnabled'] = false; window['sj_result_1efee661bdfabb84']['showMyJobsHired'] = false; window['sj_result_1efee661bdfabb84']['showSaveForSponsored'] = true; window['sj_result_1efee661bdfabb84']['showJobAge'] = true; window['sj_result_1efee661bdfabb84']['showHolisticCard'] = true; window['sj_result_1efee661bdfabb84']['showDislike'] = true; window['sj_result_1efee661bdfabb84']['showKebab'] = true; window['sj_result_1efee661bdfabb84']['showReport'] = true;</script></div></div>
</div>
</div>


<div class="tab-container">
<div class="sign-in-container result-tab"></div>
<div class="tellafriend-container result-tab email_job_content"></div>
</div>
</div>

In [20]:
# results = pd.DataFrame({'title':titles, 'salary':salaries})
# results

In [ ]:
# continue tomorrow

In [ ]:
#sdfasdf

### companies, salaries, locations, job descriptions

In [8]:
# bs = soup.find_all('span', attrs={'class':'company'})
# for title in bs:
#     print(title.text.strip())

In [9]:
# money = []
# bs = soup.find_all("span","salary no-wrap")
# for title in bs:
#     if len(title.text) >3  :
#         money.append(title.text.strip())
#     else:
#         money.append('0')

In [10]:
# bs = soup.find_all('span', attrs={'class':'location'})
# for title in bs:
#     print(title.text.strip())

In [11]:
# bs = soup.find_all("div",{"class":"summary"})
# for title in bs:
#     print(title.text.strip())